In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, String, Float, DATE
import pymssql
from datetime import date, datetime
import matplotlib.pyplot as plt
import os
from dotenv import load_dotenv
from empiricaldist import Cdf
import seaborn as sns

In [2]:
# ! heroku config:get DATABASE_URL 

In [11]:
pd.set_option('display.max_columns', 10)

env_path = os.path.join(r'/home/emad/code/emadam/glassdoor/glassdoor/',
                        'postgres_login.env')
env_path

'/home/emad/code/emadam/glassdoor/glassdoor/postgres_login.env'

In [12]:
if os.path.exists(env_path):
    load_dotenv(env_path)
DATABASE = os.getenv('database')
USERNAME = os.getenv('username')
PASSWORD = os.getenv('password')
HOST = os.getenv('host')

In [13]:
DATABASE, USERNAME, PASSWORD, HOST

('dbbib70qjmd58r',
 'svlqmfjeeogcqe',
 'f5e3f6a69bffad5b5411eafb5115fd6accf7eef9a866ca36d70df23b0595eb0e',
 'ec2-52-18-116-67.eu-west-1.compute.amazonaws.com')

In [14]:
df = pd.read_csv(
    r'/home/emad/code/emadam/glassdoor/glassdoor/glassdoor/data/data_db.csv')
df.head()

,Job Title,Company,Rank,Location,Salary,Ad Date,Min_Salary,Max_Salary,Seniority
0,Data Analyst,Cleanaway,3.5,Melbourne,NaN,2021-08-13,NaN,NaN,NaN
1,Data Analyst,ALSTOM,3.9,West Melbourne,NaN,2021-08-13,NaN,NaN,NaN
2,Data Analyst,Head Office St Kilda Rd,3.1,Melbourne,NaN,2021-08-19,NaN,NaN,NaN
3,Data Analyst,Alstom,3.9,West Melbourne,NaN,2021-08-20,NaN,NaN,NaN
4,Data Analyst,Ericsson,4.1,Melbourne,NaN,2021-08-20,NaN,NaN,NaN


In [15]:
engine = create_engine(
    f"postgresql://{USERNAME}:{PASSWORD}@{HOST}:5432/{DATABASE}")

In [16]:
df.to_sql("job_data", engine, if_exists='append', index=False)

OperationalError: (psycopg2.OperationalError) connection to server at "ec2-52-18-116-67.eu-west-1.compute.amazonaws.com" (52.18.116.67), port 5432 failed: FATAL:  role "svlqmfjeeogcqe" is not permitted to log in

(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [ ]:
class scraper():

    def __init__(self, soup):
        self.soup = soup
        pass

    def extract_job_title_from_result(soup):
        jobs = []
        for a in soup.find_all(name="a", attrs={"data-test": "job-link"}):
            for spans in a.find_all(name="span"):
                jobs.append(spans.text)
        return jobs

    def extract_company_name_from_result(soup):
        coname = []
        for div in soup.find_all(name="div", attrs={"class": "e1rrn5ka0"}):
            for div in div.find_all(name="div", attrs={"class": "d-flex justify-content-between align-items-start"}):
                for a in div.find_all(name="a"):
                    for spans in a.find_all(name="span"):
                        coname.append(spans.text)
        return coname

    def extract_company_rate_from_result(soup):
        corate = []
        for div in soup.find_all(name="div", attrs={"class": "e1rrn5ka1"}):
            if div.find_all(name="span", attrs={"class": "e1cjmv6j0"}):
                for spans in div.find_all(name="span", attrs={"class": "e1cjmv6j0"}):
                    corate.append(spans.text)
            else:
                corate.append(np.nan)
        return corate

    def extract_company_location_from_result(soup):
        coloc = []
        for div in soup.find_all(name="div", attrs={"class": "e1rrn5ka2"}):
            for spans in div.find_all(name="span"):
                coloc.append(spans.text)
        return coloc

    def extract_company_salary_from_result(soup):
        cosal = []
        for div in soup.find_all(name="div", attrs={"class": "e1rrn5ka0"}):
            if div.find_all(name="div", attrs={"class": "e1rrn5ka3"}):
                for spans in div.find_all(name="span", attrs={"data-test": "detailSalary"}):
                    cosal.append(spans.text)
            else:
                cosal.append(np.nan)
        return cosal

    def extract_job_age_from_result(soup):
        jobage = []
        for div in soup.find_all(name='div', attrs={"class": ["e1rrn5ka2", "e1rrn5ka3"]}):
            for age in div.find_all(name='div', attrs={"data-test": "job-age"}):
                result = age.text
                result = result.replace('24h', '1d')
                result = result.replace('d', '')
                result = result.replace('30+', '31')
                t_dif = np.timedelta64(result, 'D')
                if t_dif < np.timedelta64(31, 'D'):
                    ad_date = np.datetime64(date.today()) - t_dif
                    ad_date = ad_date.astype(datetime)
                    jobage.append(ad_date)
                else:
                    jobage.append(np.nan)
        return jobage

    def job_seniority(job):
        if job.find('Senior') != -1:
            return 'Senior'
        if job.find('Junior') != -1:
            return 'Junior'
        if job.find('Entry level') != -1:
            return 'Entry level'
        if job.find('Graduate') != -1:
            return 'Graduate'
        if job.find('Manager') != -1:
            return 'Manager'
        if job.find('Internship') != -1:
            return 'Internship'
        else:
            return np.nan

    def extract_job_link_from_result(soup):
        joblink = []
        for div in soup.find_all(name="div", attrs={"class": "e1rrn5ka0"}):
            for a in div.find_all(name='a', href=True):
                joblink.append('glassdoor.com.au'+a['href'])
            else:
                joblink.append(np.nan)
        return joblink

In [ ]:
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) "
                         "Chrome/92.0.4515.159 Safari/537.36"}
URL = f'https://www.glassdoor.com.au/Job/melbourne-junior-data-analyst-jobs-SRCH_IL.0,9_IC2264754_KO10,29.htm'
resp = requests.get(URL, headers=headers)
# specifying a desired format of page using the html parser
soup = BeautifulSoup(resp.text, "html.parser")

In [ ]:
job_title = scraper.extract_job_title_from_result(soup)
co_name = scraper.extract_company_name_from_result(soup)
co_rate = scraper.extract_company_rate_from_result(soup)
co_loc = scraper.extract_company_location_from_result(soup)
co_sal = scraper.extract_company_salary_from_result(soup)
job_age = scraper.extract_job_age_from_result(soup)
# job_link = scraper.extract_job_link_from_result(soup)

In [ ]:
# , job_link))
data = list(zip(job_title, co_name, co_rate, co_loc, co_sal, job_age))
job_data = pd.DataFrame(data)
job_data = job_data.rename(columns={0: 'Job Title', 1: 'Company', 2: 'Rank',
                           3: 'Location', 4: 'Salary', 5: 'Ad Date'})  # ,6: 'Hyperlink'})
job_data['Ad Date'] = pd.to_datetime(job_data['Ad Date'])

In [ ]:
job_data.info()

In [ ]:
job_data.to_sql("job_data", engine, if_exists='append', index=False)

In [9]:
jobs_stored = pd.read_sql("job_data", engine)

In [10]:
jobs_stored.head()

,Job Title,Company,Rank,Location,Salary,Ad Date
0,Junior Test Analyst,FinXL,3.4,Melbourne,None,2022-03-19
1,Junior Commercial Analyst,Coles Careers,3.6,Hawthorn East,"A$120,000 - A$161,250 (Glassdoor Est.)",2022-03-21
2,Data Analyst,Brunel,3.8,Docklands,A$47 Per Hour(Employer Est.),NaT
3,Senior Data and Insights Analyst,Open Universities Australia,3.8,Docklands,None,2022-03-23
4,Junior Financial Analyst (Graduate),BMYG Corporate Pty Ltd,None,Box Hill,"A$55,000 (Employer Est.)",NaT


In [ ]:
jobs_stored['Ad Date'] = pd.to_datetime(jobs_stored['Ad Date'])
jobs_stored.sort_values(by=['Ad Date'], inplace=True)
jobs_stored.drop_duplicates(subset=['Job Title', 'Company', 'Location'], keep='first', inplace=True)
ad_count = jobs_stored.groupby('Ad Date').size()
jobs_stored = jobs_stored.set_index(pd.DatetimeIndex(jobs_stored['Ad Date'])).sort_index()


In [ ]:
jobs_stored.info()

In [ ]:
import regex as re

In [ ]:
def salary_convert(salary):
    if salary == 0:
        return np.nan
    if salary < 1000:
        return salary * 1788
    else:
        return salary

In [ ]:
jobs_stored['Min_Salary'] = jobs_stored['Salary'].str.extract(r'([0-9]+,*[0-9]+).*')
jobs_stored['Min_Salary'] = jobs_stored['Min_Salary'].str.replace(r'\,', '', regex=True).astype(float).astype(
    pd.Int64Dtype())

jobs_stored['Max_Salary'] = jobs_stored['Salary'].str.extract(r'[0-9]+,*[0-9]+.*?([0-9]+,*[0-9]+)')
jobs_stored['Max_Salary'] = jobs_stored['Max_Salary'].str.replace(r'\,', '', regex=True).astype(float).astype(
    pd.Int64Dtype())

jobs_stored['Min_Salary'] = jobs_stored['Min_Salary'].fillna(value=0)
jobs_stored_min = jobs_stored.apply(lambda x: salary_convert(x['Min_Salary']), axis=1)
jobs_stored['Min_Salary'] = pd.DataFrame(jobs_stored_min)

jobs_stored['Max_Salary'] = jobs_stored['Max_Salary'].fillna(value=0)
jobs_stored_max = jobs_stored.apply(lambda x: salary_convert(x['Max_Salary']), axis=1)
jobs_stored['Max_Salary'] = pd.DataFrame(jobs_stored_max)

jobs_stored[['Salary', 'Min_Salary', 'Max_Salary']]

In [ ]:
jobs_stored['Seniority'] = jobs_stored['Job Title'].apply(lambda x: 'Senior' if x.find('Senior') != -1 else
('Junior' if x.find('Junior') != -1 else ('Entry Level' if x.find('Entry level') != -1 else
                                          ('Graduate' if x.find('Graduate') != -1 else
                                           ('Manager' if x.find('Manager') != -1 else
                                            ('Internship' if x.find('Internship') != -1 else np.nan))))))
jobs_stored.dropna(subset=['Ad Date'], how='all', inplace=True)
jobs_stored[['Job Title', 'Seniority']]

In [ ]:
min_sal = np.array(jobs_stored['Min_Salary'].dropna())
sal_replicates = np.empty(10000)
for i in range(10000):
    sal_samples = np.random.choice(min_sal, len(min_sal))
    sal_replicates[i] = np.mean(sal_samples)
con_interval = np.percentile(sal_replicates, [2.5, 97.5])
con_interval[0] = ('{:.1f}'.format(con_interval[0]))
con_interval[1] = ('{:.1f}'.format(con_interval[1]))
print("Minimum Salary 95% Confidence Interval = " + str(con_interval))

In [ ]:
jobs_stored['Ad Date'].sort_values(ascending=False)

In [ ]:
plt.style.use('seaborn-whitegrid')
sns.set()
fig, ax = plt.subplots(2, 2)
fig.set_size_inches(16, 11)
min_salary = jobs_stored['Min_Salary']
before_Date = jobs_stored['Ad Date'] < pd.to_datetime('2021-10-15')
ax[0, 0].plot(Cdf.from_seq(min_salary[before_Date].dropna()),
              label='Before 2021 October 15')
ax[0, 0].plot(Cdf.from_seq(min_salary[~before_Date].dropna()),
              label='After 2021 October 15')
x_min = np.sort(jobs_stored['Min_Salary'].dropna())
y_min = np.arange(1, len(x_min) + 1) / len(x_min)
x_max = np.sort(jobs_stored['Max_Salary'].dropna())
y_max = np.arange(1, len(x_max) + 1) / len(x_max)
pct_list = np.array([25, 50, 75])
maxpct_val = np.percentile(jobs_stored['Max_Salary'].dropna(), pct_list)
minpct_val = np.percentile(jobs_stored['Min_Salary'].dropna(), pct_list)
ax[0, 0].set_ylabel('CDF')
ax[0, 0].set_title(
    'Distribution of minimum salary of "Data Analyst" jobs on Glassdoor')
ax[0, 0].legend()
ax[0, 0].set_xlabel('Estimated salary')

ax[0, 1].plot(x_min, y_min, marker='.', linestyle='none',
              color='r', label='Minimum salary')
ax[0, 1].plot(x_max, y_max, marker='.', linestyle='none',
              color='b', label='Maximum salary')
ax[0, 1].plot(maxpct_val, pct_list / 100, marker='^', linestyle='none', color='c',
              label='25th, 50th and 75th Percentile')
ax[0, 1].plot(minpct_val, pct_list / 100, marker='^', linestyle='none', color='k',
              label='25th, 50th and 75th Percentile')
ax[0, 1].annotate('Mean:', xy=(jobs_stored['Min_Salary'].mean().astype('int64'), 0.5), xytext=(40000, 0.9),
                  arrowprops=dict(arrowstyle="fancy", facecolor='green', connectionstyle="angle3,angleA=0,angleB=-90"), )
ax[0, 1].set_ylabel('ECDF')
ax[0, 1].set_title(
    'Distribution of min and max salary of "Data Analyst" on Glassdoor')
ax[0, 1].legend()
ax[0, 1].set_xlabel('Estimated salary')

ax[1, 0].bar(jobs_stored.index.unique(), ad_count, linestyle='None', color='r')
ax[1, 0].figure.canvas.draw()
ax[1, 0].tick_params(axis='x', which='major', rotation=20, direction='inout', length=6, width=2,
                     color='k', labelcolor='royalblue')
ax[1, 0].set_xlabel('Date of Advertisement', labelpad=0.0, color='magenta')
ax[1, 0].set_ylabel('Number of Ads', color='purple')
ax[1, 0].set_title(
    '\'Data Analyst Job\' Advertisements in Glassdoor website', color='limegreen')

ax[1, 1].pie(jobs_stored['Seniority'].value_counts(), labels=jobs_stored['Seniority'].dropna().unique(),
             normalize=True, autopct='%1.1f%%', shadow=True, startangle=0)
ax[1, 1].set_title('Job Ads seniority level count')
# plt.tight_layout()
fig.savefig("glassdoor" + np.datetime64(date.today()).astype('str') + ".png")